In [20]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer

In [21]:
#import the data
train_x = pd.read_csv("train_final.csv")
# train_y = pd.read_csv("TrainingSetLabels.csv")
test_x = pd.read_csv("test_final.csv")
data = pd.read_csv("names.csv")

In [22]:
# # set ID as index for train x and y 
train_x = train_x.set_index(['id'])
# train_y = train_y.set_index(['id'])

In [24]:
#create x/y split for trianing 
train_y = pd.DataFrame(train_x['status_group'])
train_x = train_x.drop(['status_group'], axis = 1)

In [25]:
#save test id and drop 
test_id = pd.DataFrame(test_x['id'])
test_x = test_x.drop(['id'], axis = 1)

In [26]:
#Imputing missing values for both train and test
train_x.fillna(-999, inplace=True)
test_x.fillna(-999,inplace=True)
# train_x.isnull().sum()
# test_x.isnull().sum()

In [27]:
#convert output status group to 1,2,3 
 
replace_map = {"non functional":3, 
              "functional needs repair":2,
              "functional":1}

train_y['status_group code'] = train_y['status_group'].replace(replace_map)

In [28]:
#subset data train 
# names = data.name.tolist()
# train_x = train_x[names]

# train_x.dtypes
#train_x.isnull().sum()

In [ ]:
#set data type for non int numbers
train_x = train_x.astype({"amount_tsh": int})
test_x = test_x.astype({"amount_tsh": int})

In [29]:
#split data just for testing 
X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y['status_group code'], train_size=0.6, random_state=42)
#Identify cat features for model
categorical_features_indices = np.where(train_x.dtypes != np.float)[0]

In [30]:
## test model 
# model_test = CatBoostClassifier(depth = 10, 
#                                iterations = 500,
#                                l2_leaf_reg = 2,
#                                leaf_estimation_iterations = 10,
#                                loss_function = 'MultiClass',
#                                random_seed = 42)
                   
# model_test.fit(X_train, Y_train ,cat_features=categorical_features_indices)
# preds_class_full = model_test.predict(X_test)
# accuracy_score(preds_class_full,Y_test)



In [31]:
# from sklearn.metrics import confusion_matrix

# confusion_matrix(preds_class_full,Y_test)

In [32]:
#grid search
clf = CatBoostClassifier()
params = {'iterations': [1],
          'learning_rate': [.05, .1,.5, 1],
          'depth': [10],
          'loss_function': ['MultiClass'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }

params = {'depth':[6,8,10],
          'iterations':[500],
          'learning_rate':[0.01,0.1,0.2,0.5],
          'logging_level':['Silent'],
          'l2_leaf_reg':[3,1,5,10,100]}


scorer = make_scorer(accuracy_score)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)



In [ ]:
clf_grid.fit(X_train, Y_train, cat_features=categorical_features_indices)
best_param = clf_grid.best_params_
best_param

In [ ]:
# #full model from grid search parameters

# categorical_features_indices = np.where(train_x.dtypes != np.float)[0]
# #full model
# model_full = CatBoostClassifier(depth = 10, 
#                                iterations = 1000,
#                                 learning_rate =   .1,
#                                leaf_estimation_iterations = 10,
#                                loss_function = 'MultiClass',
#                                random_seed = 42)
                   
# model_full.fit(train_x, train_y['status_group code'] ,cat_features=categorical_features_indices)



In [ ]:
#send best param to 
pd.DataFrame.from_dict(best_param, orient='index').to_csv('best_param.csv')

In [ ]:
#create submission file
#Create predictions
preds_class_full = model_full.predict(test_x)
#submision
preds_class_full = pd.DataFrame(preds_class_full)
submission = pd.concat([test_id, preds_class_full], axis=1)
submission.columns = ['id', 'status_group']
replace_map2 = {3:"non functional", 
              2:"functional needs repair",
              1:"functional"}

submission['status_group'] = submission['status_group'].replace(replace_map2)
submission.to_csv("submision.csv", index=False)